In [1]:
import femr.models.transformer
import torch
import femr.models.tokenizer
import femr.models.processor
import datetime
from collections import defaultdict
import json
model_name = "StanfordShahLab/clmbr-t-base"

In [2]:
# !pip install xformers

In [3]:
tokenizer = femr.models.tokenizer.FEMRTokenizer.from_pretrained(model_name)
batch_processor = femr.models.processor.FEMRBatchProcessor(tokenizer)

# Load model
model = femr.models.transformer.FEMRModel.from_pretrained(model_name)

dictionary.msgpack:   0%|          | 0.00/6.84M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/567M [00:00<?, ?B/s]

In [12]:
model.transformer.layers[0].norm

RMSNorm()

In [14]:
model.transformer.layers[0].input_proj

Linear(in_features=768, out_features=5376, bias=True)

In [16]:
model.transformer.layers[0].output_proj

Linear(in_features=3840, out_features=768, bias=True)

In [17]:
model

FEMRModel(
  (transformer): FEMRTransformer(
    (in_norm): RMSNorm()
    (out_norm): RMSNorm()
    (embed): Embedding(65536, 768)
    (layers): ModuleList(
      (0-11): 12 x FEMREncoderLayer(
        (norm): RMSNorm()
        (input_proj): Linear(in_features=768, out_features=5376, bias=True)
        (output_proj): Linear(in_features=3840, out_features=768, bias=True)
      )
    )
  )
  (task_model): CLMBRTaskHead(
    (final_layer): Linear(in_features=768, out_features=8192, bias=True)
  )
)

In [3]:
import inspect
source_file = inspect.getsourcefile(femr.models.processor)
source_file

'/home/ubuntu/anaconda3/lib/python3.11/site-packages/femr/models/processor.py'

In [4]:
tokenizer.dictionary['vocab'][0]['code_string']
tokenizer.dictionary['vocab'][0]['weight']

-0.18811663446051943

In [5]:
dict = defaultdict()
for i in range(len(tokenizer.dictionary['vocab'])):
    dict[tokenizer.dictionary['vocab'][i]['code_string']] = tokenizer.dictionary['vocab'][i]['weight']

In [6]:
with open('femr_vocab.json', 'w') as f:
    json.dump(dict, f)

In [7]:
fake_event = [{
        'time': datetime.datetime(i+10, 5, 8),
        'measurements': [
            {'code': 'SNOMED/184099003'},
            {'code': 'Visit/IP'},
        ],
    } for i in range(5000)]

fake_patient = {'patient_id':30, 'events': fake_event}

In [8]:
patient_sample = {
    'patient_id': 30,
    'events': [{
        'time': datetime.datetime(2011, 5, 8),
        'measurements': [
            {'code': 'SNOMED/184099003'},
            {'code': 'Visit/IP'},
        ],
    },
    {
        'time': datetime.datetime(2012, 6, 9),
        'measurements': [
            {'code': 'Visit/OP'},
            {'code': 'SNOMED/3950001'}
        ],
    }]
}

patient_sample_0 = {
    'patient_id': 30,
    'events': [{
        'time': datetime.datetime(2011, 5, 8),
        'measurements': [
            {'code': 'SNOMED/184099003'},
            {'code': 'Visit/IP'},
        ],
    }]
}

In [16]:
# len(patient_sample_1['events'])

In [10]:
m = [{'code': 'SNOMED/3950001'} for i in range(10)]
m = [{'code': 'Visit/OP'}] + m

patient_sample_2 = {
    'patient_id': 30,
    'events': [{
        'time': datetime.datetime(2011, 5, 8),
        'measurements': m,
    }]
}

In [15]:
# raw_batch = batch_processor.convert_patient(fake_patient, tensor_type="pt")
# batch = batch_processor.collate([raw_batch])

# _, result = model(**batch)
# result

In [12]:
patient_sample_1 = {
    'patient_id': 30,
    'events': [{
        'time': datetime.datetime(2011, 5, 8),
        'measurements': [
            {'code': 'SNOMED/184099003'},
            {'code': 'Visit/IP'},
            {'code': 'Gender/M'}
        ],
    },
    {
        'time': datetime.datetime(2012, 6, 9),
        'measurements': [
            {'code': 'Visit/OP'},
            {'code': 'SNOMED/3950001'},
        ],
    }, 
    {
        'time': datetime.datetime(2018, 6, 9),
        'measurements': [
            {'code': 'Visit/OP'},
            {'code': 'SNOMED/3950001'},
        ],
    }, 
    {
        'time': datetime.datetime(2018, 7, 9),
        'measurements': [
            {'code': 'Visit/OP'},
            {'code': 'SNOMED/3950001'},
        ],
    }, 
    {
        'time': datetime.datetime(2020, 6, 9, 9),
        'measurements': [
            {'code': 'SNOMED/3950001'},
            {'code': 'Visit/OP'}
        ],
    }, 
    {
        'time': datetime.datetime(2020, 6, 9, 10),
        'measurements': [
            {'code': 'Visit/IP'},
            {'code': 'SNOMED/3950001'},
        ],
    }]
}
raw_batch_2 = batch_processor.convert_patient(patient_sample_1, tensor_type="pt")
batch_2 = batch_processor.collate([raw_batch_2])

_, result_2 = model(**batch_2)

In [42]:
patient_sample_1['events']['time']

TypeError: list indices must be integers or slices, not str

In [37]:
for i in batch_2['batch']['transformer']['tokens'].numpy()[0]:
    print(tokenizer.dictionary['vocab'][i]['code_string'])

SNOMED/184099003
Visit/IP
Gender/M
Visit/OP
Visit/OP
Visit/OP
Visit/OP
Visit/IP


In [40]:
raw_batch_2

{'num_patients': tensor(1),
 'num_indices': tensor(0),
 'patient_ids': tensor([30, 30, 30, 30, 30, 30, 30, 30]),
 'offsets': tensor([0]),
 'transformer': {'valid_tokens': tensor([True, True, True, True, True, True, True, True]),
  'ages': tensor([   0.0000,    0.0000,    0.0000,  398.0000, 2589.0000, 2619.0000,
          3320.3750, 3320.4167]),
  'normalized_ages': tensor([-1.3066, -1.3066, -1.3066,  1.1699, 14.8047, 14.9922, 19.3594, 19.3594]),
  'timestamps': tensor([1304812800, 1304812800, 1304812800, 1339200000, 1528502400, 1531094400,
          1591693200, 1591696800]),
  'patient_lengths': array([8]),
  'label_indices': tensor([], dtype=torch.int64),
  'tokens': tensor([  0, 247,   3,   4,   4,   4,   4, 247])}}

In [44]:
len(patient_sample_1['events'])

6

In [ ]:
raw_batch_1 = batch_processor.convert_patient(patient_sample_1, tensor_type="pt")
batch_1 = batch_processor.collate([raw_batch_1])

_, result_1 = model(**batch_1)

In [ ]:
result_1

{'timestamps': tensor([1304812800, 1304812800, 1339200000]),
 'patient_ids': tensor([30, 30, 30]),
 'representations': tensor([[ 0.0461,  0.6623, -0.6115,  ...,  0.0732, -1.2741, -0.3073],
         [ 0.6681, -1.9489, -2.2254,  ..., -2.2320, -0.4854,  1.0614],
         [-0.8006, -1.2952, -1.5904,  ..., -1.3272,  1.9747,  0.6058]],
        grad_fn=<ViewBackward0>)}

In [ ]:
batch_1['batch']

{'num_patients': tensor([1]),
 'num_indices': tensor([0]),
 'patient_ids': tensor([[30, 30, 30, 30]]),
 'offsets': tensor([[0]]),
 'transformer': {'valid_tokens': tensor([[True, True, True, True]]),
  'ages': tensor([[  0.,   0., 398., 398.]]),
  'normalized_ages': tensor([[-1.3066, -1.3066,  1.1699,  1.1699]]),
  'timestamps': tensor([[1304812800, 1304812800, 1339200000, 1339200000]]),
  'patient_lengths': array([[4]]),
  'label_indices': tensor([], size=(1, 0), dtype=torch.int64),
  'tokens': tensor([[   0,  247,    0, 2656]])}}

In [ ]:
batch_1

{'batch': {'num_patients': tensor([1]),
  'num_indices': tensor([0]),
  'patient_ids': tensor([[30, 30, 30, 30, 30]]),
  'offsets': tensor([[0]]),
  'transformer': {'valid_tokens': tensor([[True, True, True, True, True]]),
   'ages': tensor([[  0.,   0., 398., 398., 398.]]),
   'normalized_ages': tensor([[-1.3066, -1.3066,  1.1699,  1.1699,  1.1699]]),
   'timestamps': tensor([[1304812800, 1304812800, 1339200000, 1339200000, 1339200000]]),
   'patient_lengths': array([[5]]),
   'label_indices': tensor([], size=(1, 0), dtype=torch.int64),
   'tokens': tensor([[   0,  247,    4,    0, 2656]])}}}